In [33]:
import numpy as np
import pyvista as pv
import cdd
from scipy.spatial import ConvexHull
from pyvista.core.pointset import PolyData
from sklearn.decomposition import PCA
import tensorflow_datasets as tfds

In [24]:
# take one cube
cube1: PolyData = pv.Cube()
# take the same cube but translate it 
cube2: PolyData = pv.Cube() 
cube2 = cube2.translate((0.5, 0.5, 0.5))

# plot 
pltr = pv.Plotter(window_size=[512,512])
pltr.add_mesh(cube1)
pltr.add_mesh(cube2)
pltr.show()

Widget(value="<iframe src='http://localhost:41431/index.html?ui=P_0x7f3e7400af20_14&reconnect=auto' style='wid…

In [56]:
pts1 = cube1.points
pts2 = cube2.points

pts1 = np.array([
    [0, 0, 0, 0.5],
    [0, 0, 1, 0.5],
    # [0, 1, 0, 0.5],
    # [0, 1, 1, 0.5],
    # [1, 0, 0, 0.5],
    # [1, 0, 1, 0.5],
    # [1, 1, 0, 0.5],
    # [1, 1, 1, 0.5],
    # [0.5, 0.5, 0.5, 0.5]
])

pts2 = np.array([
    [0, 0, 0, 0.8],
    [0, 0, 1, 0.8],
    [0, 1, 0, 0.8],
    [0, 1, 1, 0.8],
    [1, 0, 0, 0.8],
    [1, 0, 1, 0.8],
    [1, 1, 0, 0.8],
    [1, 1, 1, 0.8]
]) + 0.3

# make the V-representation of the first cube; you have to prepend
# with a column of ones
v1 = np.column_stack((np.ones(len(pts1)), pts1))

mat = cdd.Matrix(v1, number_type='float') # use fractions if possible
mat.rep_type = cdd.RepType.GENERATOR
poly1 = cdd.Polyhedron(mat)

# make the V-representation of the second cube; you have to prepend
# with a column of ones
v2 = np.column_stack((np.ones(len(pts2)), pts2))

mat = cdd.Matrix(v2, number_type='float')
mat.rep_type = cdd.RepType.GENERATOR
poly2 = cdd.Polyhedron(mat)

# H-representation of the first cube
h1 = poly1.get_inequalities()

# H-representation of the second cube
h2 = poly2.get_inequalities()

# join the two sets of linear inequalities; this will give the intersection
hintersection = np.vstack((h1, h2))

# make the V-representation of the intersection
mat = cdd.Matrix(hintersection, number_type='float')
mat.rep_type = cdd.RepType.INEQUALITY
polyintersection = cdd.Polyhedron(mat)

# get the vertices; they are given in a matrix prepended by a column of ones
vintersection = polyintersection.get_generators()

print(vintersection)
    
# get rid of the column of ones
ptsintersection = np.array([
    vintersection[i][1:] for i in range(len(vintersection))
])

print(ptsintersection)

# these are the vertices of the intersection; it remains to take
# the convex hull
ConvexHull(ptsintersection)

V-representation
begin
 9 5 real
  1  1.300000000E+00  3.000000000E-01  1  1.100000000E+00
  1  1.300000000E+00  3.000000000E-01  3.000000000E-01  1.100000000E+00
  1  3.000000000E-01  3.000000000E-01  1  1.100000000E+00
  1  3.000000000E-01  3.000000000E-01  3.000000000E-01  1.100000000E+00
  1  3.000000000E-01  1.300000000E+00  3.000000000E-01  1.100000000E+00
  1  3.000000000E-01  1.300000000E+00  1  1.100000000E+00
  1  1.300000000E+00  1.300000000E+00  3.000000000E-01  1.100000000E+00
  1  1.300000000E+00  1.300000000E+00  1  1.100000000E+00
  0  0  0  0  1
end
[[1.3 0.3 1.  1.1]
 [1.3 0.3 0.3 1.1]
 [0.3 0.3 1.  1.1]
 [0.3 0.3 0.3 1.1]
 [0.3 1.3 0.3 1.1]
 [0.3 1.3 1.  1.1]
 [1.3 1.3 0.3 1.1]
 [1.3 1.3 1.  1.1]
 [0.  0.  0.  1. ]]


In [4]:
def get_inequality(v):
    """
    Returns the inequality representation of a vector considering it as polyhedron
    """
    v = np.column_stack((np.ones(len(v)), v))
    mat = cdd.Matrix(v, number_type='float')
    mat.rep_type = cdd.RepType.GENERATOR
    poly = cdd.Polyhedron(mat)
    return poly.get_inequalities()
    
def get_convex_hull(v):
    """
    Returns the convex hull of a vector considering it as polyhedron
    """
    hull = ConvexHull(v)
    return hull


def convex_hull_intersection(inequality1, inequality2):
    """
    Returns the convex hull of the intersection of two polyhedra, given as
    the inequalities defining them.
    you can get the inequalities from vectors like below:
    ```python
    v1 = np.column_stack((np.ones(len(pts1)), pts1))
    mat = cdd.Matrix(v1, number_type='float')
    mat.rep_type = cdd.RepType.GENERATOR
    poly1 = cdd.Polyhedron(mat)
    inequality1 = poly1.get_inequalities()
    ```
    """
    # convert to the H-representation
    h = np.vstack((inequality1, inequality2))
    mat = cdd.Matrix(h, number_type='float')
    mat.rep_type = cdd.RepType.INEQUALITY
    poly = cdd.Polyhedron(mat)

    # get the vertices of the convex hull
    mat = poly.get_generators()
    mat = np.array(mat)
    if len(mat) == 0: return 0
    if mat.shape[0] < mat.shape[1]: return 0
    hull = ConvexHull(mat[:, 1:])
    return hull

In [40]:
activations = np.load('activationSummary_imagenette.npz')
activation_labels = np.load('labels_imagenette.npz')

print(activations.files)
print(activation_labels.files)
print(activations['block1_conv1'].shape)
print(activation_labels['labels'].shape)

['block1_conv1', 'block1_conv2', 'block2_conv1', 'block2_conv2', 'block3_conv1', 'block3_conv2', 'block3_conv3', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block5_conv1', 'block5_conv2', 'block5_conv3', 'flatten', 'fc1', 'fc2', 'predictions']
['labels']
(9469, 64)
(9469,)


In [41]:
label1 = 0
label2 = 1

layer = 'block5_conv3'
activations1 = activations[layer][activation_labels['labels'] == label1]
activations2 = activations[layer][activation_labels['labels'] == label2]
print(activations1.shape)
print(activations2.shape)

activation1_inequality = get_inequality(activations1)
activation2_inequality = get_inequality(activations2)

print(activation1_inequality)
print(activation2_inequality)


(963, 512)
(955, 512)
H-representation
begin
 0 513 real
end
H-representation
begin
 0 513 real
end


In [42]:
from rich.table import Table
from rich.console import Console
table = Table()
table.add_column("Class")
table.add_column("Mean")
table.add_column("Std")

for label in range(len(set(activation_labels['labels']))):
    activation = activations[layer][activation_labels['labels'] == label]
    table.add_row(str(labels.names[label]), str(activation.mean(axis=1).mean().round(decimals=0).astype(int)), str(activation.std(axis=1).mean().round(decimals=0).astype(int)))
    
console = Console()
console.print(table)

┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━┓
┃ Class                 ┃ Mean ┃ Std ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━┩
│ tench.n.01            │ 69   │ 98  │
│ english_springer.n.01 │ 70   │ 104 │
│ cassette_player.n.01  │ 53   │ 84  │
│ chain_saw.n.01        │ 55   │ 72  │
│ church.n.02           │ 45   │ 79  │
│ french_horn.n.01      │ 67   │ 98  │
│ garbage_truck.n.01    │ 58   │ 88  │
│ gas_pump.n.01         │ 61   │ 86  │
│ golf_ball.n.01        │ 48   │ 94  │
│ parachute.n.01        │ 43   │ 71  │
└───────────────────────┴──────┴─────┘

In [39]:
from nltk.corpus import wordnet as wn
ds, info = tfds.load(
    'imagenette/320px-v2',
    shuffle_files=False,
    with_info=True,
    as_supervised=True,
    batch_size=None,
)
labels = tfds.features.ClassLabel(
    names=list(map(lambda l: wn.synset_from_pos_and_offset(
        l[0], int(l[1:])).name(), info.features['label'].names))
)

labels.names

['tench.n.01',
 'english_springer.n.01',
 'cassette_player.n.01',
 'chain_saw.n.01',
 'church.n.02',
 'french_horn.n.01',
 'garbage_truck.n.01',
 'gas_pump.n.01',
 'golf_ball.n.01',
 'parachute.n.01']

In [39]:
mat = cdd.Matrix([[1, 1, 0], [1, 0, 1], [1, -1, 0], [1, 0, -1]])
mat.rep_type = cdd.RepType.INEQUALITY
poly = cdd.Polyhedron(mat)
# The V-representation can be printed in the usual way:
gen = poly.get_generators()
h = poly.get_inequalities()
print(gen)
print(h)


V-representation
begin
 4 3 rational
 1 1 -1
 1 1 1
 1 -1 1
 1 -1 -1
end
H-representation
begin
 4 3 rational
 1 1 0
 1 0 1
 1 -1 0
 1 0 -1
end


In [53]:
# Define a 3D array
a = np.array([
    [
        [1, 2, 3],
        [4, 5, 6],
        [1, 2, 3],
        [8, 9, 10]
    ],
    [
        [1, 2, 3],
        [4, 5, 6],
        [1, 2, 3],
        [8, 9, 10]
    ]
    ])
print(a.shape)
np.linalg.norm(a, axis=(0,1)).sum(axis=0)

(2, 4, 3)


45.45584612019144

In [38]:
mat = cdd.Matrix([[1, 1, 0, 1.7], [1, 0, 1, 1.3], [1, -1, 0, 1.0], [1, 0, -1, 1.8]])
mat.rep_type = cdd.RepType.GENERATOR
poly = cdd.Polyhedron(mat)
# The V-representation can be printed in the usual way:
gen = poly.get_generators()
h = poly.get_inequalities()
print(gen)
print(h)


V-representation
begin
 4 4 real
  1  1  0  1.700000000E+00
  1  0  1  1.300000000E+00
  1 -1  0  1
  1  0 -1  1.800000000E+00
end
H-representation
begin
 4 4 real
  1.033333333E+01  1 -1.666666667E+00 -6.666666667E+00
 -27 -7  1 20
 -3.857142857E+00 -1  1.285714286E+00  2.857142857E+00
  6.200000000E+00  2.200000000E+00 -1 -4
end


In [43]:
v = np.column_stack((np.ones(len(activations1[:, :5])), activations1[:, :10]))
mat = cdd.Matrix(v, number_type='float')
mat.rep_type = cdd.RepType.GENERATOR
poly = cdd.Polyhedron(mat)
val = poly.get_inequalities()
print(val)

RuntimeError: failed to load polyhedra
*Error: Numerical inconsistency is found.  Use the GMP exact arithmetic.

In [13]:
hull1 = get_convex_hull(activations1)
hull1

In [8]:
hull1 = get_convex_hull(activations1)
hull2 = get_convex_hull(activations2)
intersection_hull = convex_hull_intersection(activation1_inequality, activation2_inequality)

hull1

In [ ]:
hull1.vertices

In [10]:
pca = PCA(n_components=2)
activations12 = pca.fit_transform(np.concatenate([activations1, activations2]))
activations1_pca = activations12[:len(activations1)]
activations2_pca = activations12[len(activations1):]

In [12]:
activations1_pca_hull = get_convex_hull(activations1_pca)
activations2_pca_hull = get_convex_hull(activations2_pca)

activations1_pca_hull.vertices

array([904, 291, 328, 905, 584, 445,  82, 446, 477, 323], dtype=int32)